In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [3]:
dataset = pd.read_csv("/content/drive/MyDrive/projects/NLP-TheProject/Dataset/bbc_news.csv")
dataset.shape

FileNotFoundError: ignored

In [ ]:
dataset.head()

In [ ]:
dataset['category_id'] = dataset.Category.factorize()[0]
dataset.head()

In [ ]:
print(dataset['Category'].value_counts())

In [ ]:
plt.figure(figsize=(12,5))
sns.countplot(x=dataset.Category, color='yellow')
plt.title('BBC news class distribution', fontsize=16)
plt.ylabel('Class Counts', fontsize=16)
plt.xlabel('Class Label', fontsize=16)
plt.xticks(rotation='vertical');

In [ ]:
# Converting the catergory labels into numeric values using Encoder
labelEncoder = preprocessing.LabelEncoder()
dataset['category_id'] = labelEncoder.fit_transform(dataset['Category'])
print(dataset['category_id'].value_counts())

In [ ]:
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.remove_stopwords ,
           gsp.strip_short,
           gsp.stem_text
          ]

def clean_text(dataset):
    dataset = dataset.lower()
    dataset = utils.to_unicode(dataset)
    for f in filters:
        dataset = f(dataset)
    return dataset

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
example_sent = "This is a sample sentence, showing off the stop words filtration."
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(example_sent)
filtered_sentence = [w for w in word_tokens if not w in stop_words]
filtered_sentence = []
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
print(word_tokens)
print(filtered_sentence)

"""
import pandas as pd
import nltk

df = pd.read_csv("/path/to/file.csv")

start = time.time()
df["unigrams"] = df["verbatim"].apply(nltk.word_tokenize)
print "series.apply", (time.time() - start)

start = time.time()
df["unigrams2"] = df.apply(lambda row: nltk.word_tokenize(row["verbatim"]), axis=1)
print "dataframe.apply", (time.time() - start)"""

In [ ]:
dataset["clean_text"] = dataset["Text"].apply(clean_text)
dataset.head()

In [ ]:
dataset.iloc[2,1]

In [ ]:
clean_text(dataset.iloc[2,1])

In [ ]:
# Splitting the dataset into Training and Testing Data
train_text, test_text, train_labels, test_labels = train_test_split(dataset["clean_text"], dataset["category_id"], train_size=0.7, random_state=5)

print("Train x size :", train_text.shape)
print("Train y size :", train_labels.shape)
print("Test x size :", test_text.shape)
print("Test y size :", test_labels.shape)